In [155]:
import polars as pl

In [156]:
pl_df = pl.scan_parquet("filer/kjoretoyinfo_fra_2000.parquet").head(10).collect()

In [157]:
pl_df

tekn_reg_f_g,tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_merke,tekn_modell,tekn_farge,tekn_drivstoff,tekn_siste_pkk,tekn_neste_pkk,tekn_aksler_drift,tekn_sitteplasser_totalt
i64,i64,i64,i64,str,str,str,str,str,i64,i64
20211013,20211013,20211013,3900,"""VITO""","""05""","""2""",null,"""20251013""",2,3
20200129,20200129,20200129,5750,"""CADDY""","""06""","""2""","""20231205""","""20260129""",2,2
20231129,20231129,20231129,2600,"""Mustang Mach-E…","""06""","""5""",null,"""20271129""",1,5
20200127,20200127,20200127,5750,"""CADDY""","""01""","""2""",null,"""20240127""",2,2
20180731,20190130,20190130,2600,"""F-150 Platinum…","""06""","""2""","""20220503""","""20240503""",2,2
20200519,20200519,20200519,5750,"""CRAFTER""","""01""","""2""",null,"""20240519""",1,3
20200911,20200911,20220216,5150,"""CITIGO""","""01""","""5""",null,"""20240911""",1,4
20220512,20220512,20220512,5480,"""YARIS""","""04""","""1,5""",null,"""20260512""",1,5
20221104,20221104,20221104,5480,"""RAV4""","""06""","""1,5,5""",null,"""20261104""",2,5


In [158]:
"""
“tekn_reg_f_g_n” er dato for førstegangsregistrering i Norge. Konverter denne
kolonnen til en datetime. Gjør tilsvarende for “tekn_reg_eier_dato”. Vigjør dette med en "alias" så programmet ikke skal kræsje når vi executer igjen:
"""
pl_df = pl_df.with_columns([
    pl.col("tekn_reg_f_g_n").cast(str).str.strptime(pl.Datetime, "%Y%m%d").alias("tekn_reg_f_g_n_datetime"),
    pl.col("tekn_reg_eier_dato").cast(str).str.strptime(pl.Datetime, "%Y%m%d").alias("tekn_reg_eier_dato datetime"),
   
])

pl_df

tekn_reg_f_g,tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_merke,tekn_modell,tekn_farge,tekn_drivstoff,tekn_siste_pkk,tekn_neste_pkk,tekn_aksler_drift,tekn_sitteplasser_totalt,tekn_reg_f_g_n_datetime,tekn_reg_eier_dato datetime
i64,i64,i64,i64,str,str,str,str,str,i64,i64,datetime[μs],datetime[μs]
20211013,20211013,20211013,3900,"""VITO""","""05""","""2""",null,"""20251013""",2,3,2021-10-13 00:00:00,2021-10-13 00:00:00
20200129,20200129,20200129,5750,"""CADDY""","""06""","""2""","""20231205""","""20260129""",2,2,2020-01-29 00:00:00,2020-01-29 00:00:00
20231129,20231129,20231129,2600,"""Mustang Mach-E…","""06""","""5""",null,"""20271129""",1,5,2023-11-29 00:00:00,2023-11-29 00:00:00
20200127,20200127,20200127,5750,"""CADDY""","""01""","""2""",null,"""20240127""",2,2,2020-01-27 00:00:00,2020-01-27 00:00:00
20180731,20190130,20190130,2600,"""F-150 Platinum…","""06""","""2""","""20220503""","""20240503""",2,2,2019-01-30 00:00:00,2019-01-30 00:00:00
20200519,20200519,20200519,5750,"""CRAFTER""","""01""","""2""",null,"""20240519""",1,3,2020-05-19 00:00:00,2020-05-19 00:00:00
20200911,20200911,20220216,5150,"""CITIGO""","""01""","""5""",null,"""20240911""",1,4,2020-09-11 00:00:00,2022-02-16 00:00:00
20220512,20220512,20220512,5480,"""YARIS""","""04""","""1,5""",null,"""20260512""",1,5,2022-05-12 00:00:00,2022-05-12 00:00:00
20221104,20221104,20221104,5480,"""RAV4""","""06""","""1,5,5""",null,"""20261104""",2,5,2022-11-04 00:00:00,2022-11-04 00:00:00


In [159]:
#Gjør tilsvarende for “neste_pkk” og “tekn_reg_eier_dato”
pl_df = pl_df.with_columns([
    pl.col("tekn_neste_pkk").str.strptime(pl.Datetime, "%Y%m%d").alias("tekn_neste_pkk"),
   
])

pl_df

tekn_reg_f_g,tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_merke,tekn_modell,tekn_farge,tekn_drivstoff,tekn_siste_pkk,tekn_neste_pkk,tekn_aksler_drift,tekn_sitteplasser_totalt,tekn_reg_f_g_n_datetime,tekn_reg_eier_dato datetime
i64,i64,i64,i64,str,str,str,str,datetime[μs],i64,i64,datetime[μs],datetime[μs]
20211013,20211013,20211013,3900,"""VITO""","""05""","""2""",null,2025-10-13 00:00:00,2,3,2021-10-13 00:00:00,2021-10-13 00:00:00
20200129,20200129,20200129,5750,"""CADDY""","""06""","""2""","""20231205""",2026-01-29 00:00:00,2,2,2020-01-29 00:00:00,2020-01-29 00:00:00
20231129,20231129,20231129,2600,"""Mustang Mach-E…","""06""","""5""",null,2027-11-29 00:00:00,1,5,2023-11-29 00:00:00,2023-11-29 00:00:00
20200127,20200127,20200127,5750,"""CADDY""","""01""","""2""",null,2024-01-27 00:00:00,2,2,2020-01-27 00:00:00,2020-01-27 00:00:00
20180731,20190130,20190130,2600,"""F-150 Platinum…","""06""","""2""","""20220503""",2024-05-03 00:00:00,2,2,2019-01-30 00:00:00,2019-01-30 00:00:00
20200519,20200519,20200519,5750,"""CRAFTER""","""01""","""2""",null,2024-05-19 00:00:00,1,3,2020-05-19 00:00:00,2020-05-19 00:00:00
20200911,20200911,20220216,5150,"""CITIGO""","""01""","""5""",null,2024-09-11 00:00:00,1,4,2020-09-11 00:00:00,2022-02-16 00:00:00
20220512,20220512,20220512,5480,"""YARIS""","""04""","""1,5""",null,2026-05-12 00:00:00,1,5,2022-05-12 00:00:00,2022-05-12 00:00:00
20221104,20221104,20221104,5480,"""RAV4""","""06""","""1,5,5""",null,2026-11-04 00:00:00,2,5,2022-11-04 00:00:00,2022-11-04 00:00:00


In [160]:
"""
Join inn navnet på fargen, det vil si, join “tekn_farge” med “kode” i fargekode.csv.
Pass på at du ikke mister noen biler som vi ikke vet fargen på.
"""

# Les inn dataene fra parquet-filen
kjoretoy_df = pl.scan_parquet("filer/kjoretoyinfo_fra_2000.parquet").head(10).collect()

# Les inn de første 10 radene fra CSV-filen med semikolon som skilletegn
fargekode_df = pl.scan_csv("filer/fargekode.csv", separator=";", truncate_ragged_lines=True).head(10).collect()

kjoretoy_df = kjoretoy_df.with_columns(
    kjoretoy_df["tekn_farge"].str.strip_chars("0")
)

# Konverter "kode" til streng og gi den et annet navn
fargekode_df = fargekode_df.select([
    pl.col("*"),
    pl.col("kode").cast(pl.String).alias("fargekode")  # Konverter "kode" til streng
])

# Utfør venstre join basert på kolonnene tekn_farge og kode
joined_df = kjoretoy_df.join(
    fargekode_df,
    left_on="tekn_farge",  # Kolonnenavn i kjoretoy_df
    right_on="fargekode",  # Kolonnenavn i fargekode_df
    how="left"

)

joined_df = joined_df.with_columns(
    "beskrivelse",
    pl.when(joined_df["kode"].is_null()).then(pl.lit("Ukjent farge")).otherwise(joined_df["beskrivelse"])
)


# Vis de første 10 radene av resultatet
joined_df



tekn_reg_f_g,tekn_reg_f_g_n,tekn_reg_eier_dato,tekn_merke,tekn_modell,tekn_farge,tekn_drivstoff,tekn_siste_pkk,tekn_neste_pkk,tekn_aksler_drift,tekn_sitteplasser_totalt,kode,beskrivelse,literal
i64,i64,i64,i64,str,str,str,str,str,i64,i64,i64,str,str
20211013,20211013,20211013,3900,"""VITO""","""5""","""2""",null,"""20251013""",2,3,5,"""Grå""","""Grå"""
20200129,20200129,20200129,5750,"""CADDY""","""6""","""2""","""20231205""","""20260129""",2,2,6,"""Svart (også bl…","""Svart (også bl…"
20231129,20231129,20231129,2600,"""Mustang Mach-E…","""6""","""5""",null,"""20271129""",1,5,6,"""Svart (også bl…","""Svart (også bl…"
20200127,20200127,20200127,5750,"""CADDY""","""1""","""2""",null,"""20240127""",2,2,1,"""Hvit (også ant…","""Hvit (også ant…"
20180731,20190130,20190130,2600,"""F-150 Platinum…","""6""","""2""","""20220503""","""20240503""",2,2,6,"""Svart (også bl…","""Svart (også bl…"
20200519,20200519,20200519,5750,"""CRAFTER""","""1""","""2""",null,"""20240519""",1,3,1,"""Hvit (også ant…","""Hvit (også ant…"
20200911,20200911,20220216,5150,"""CITIGO""","""1""","""5""",null,"""20240911""",1,4,1,"""Hvit (også ant…","""Hvit (også ant…"
20220512,20220512,20220512,5480,"""YARIS""","""4""","""1,5""",null,"""20260512""",1,5,4,"""Grønn (også gr…","""Grønn (også gr…"
20221104,20221104,20221104,5480,"""RAV4""","""6""","""1,5,5""",null,"""20261104""",2,5,6,"""Svart (også bl…","""Svart (også bl…"


In [161]:
#Lag en kolonne “elbil” som er sann hvis drivstofftypen er kun elektrisk, false ellers.

joined_df = joined_df.with_columns(
    "elbil",
    pl.when(joined_df["tekn_drivstoff"] == "5").then(True).otherwise(False)
)
joined_df

ColumnNotFoundError: elbil

Error originated just after this operation:
DF ["tekn_reg_f_g", "tekn_reg_f_g_n", "tekn_reg_eier_dato", "tekn_merke"]; PROJECT */14 COLUMNS; SELECTION: "None"